## Weave module deps visualization

Visualize dependencies within the Weave core modules, so we can untangle circular imports and get organized.

If the output graph is blank, there is an error somewhere. Look for the "MISSING" prints inline in this notebook to diagnose. Either our hacky import line parsing didn't work, or you need to add more sub-modules to "modules +=".

In [ ]:
#!pip install dagre_py

In [ ]:
import pathlib

from dagre_py.core import plot

In [ ]:
# Load non-test python files from 'weave/'

files = [p for p in pathlib.Path("../weave").iterdir() if p.name.endswith(".py")]
files = [p for p in files if not p.name.startswith("test_")]
files = [p for p in files if not p.name == "conftest.py"]
len(files)

In [ ]:
def get_module_name(file):
    return file.name.split(".py")[0]


# convert files to modules
modules = [get_module_name(f) for f in files]

# add directory modules
modules += [
    "panels",
    "ecosystem",
    "ops_primitives",
    "ops_domain",
    "ops_arrow",
    "language_features",
]
# modules

In [ ]:
def get_import_component(imp: str):
    imp = imp.strip().strip("#").strip()
    # 'from .X'
    if imp[6] != " ":
        return imp.split()[1][1:].split(".")[0]
    imp = imp.split("import ")[1]
    return imp.split(" ")[0]


def get_imports(path: pathlib.Path):
    top_level_imports = []
    inline_imports = []
    for line in open(path).read().split("\n"):
        if line.startswith("from ."):
            top_level_imports.append(get_import_component(line))
        elif "from ." in line:
            inline_imports.append(get_import_component(line))
    return top_level_imports, inline_imports

In [ ]:
nodes = [{"label": m} for m in modules]
# nodes

In [ ]:
edges = []
edge_num = 0
for p, m in zip(files, modules):
    top_level_imports, inline_imports = get_imports(p)
    for imp in top_level_imports:
        edges.append({"source": m, "target": imp})
    for imp in inline_imports:
        edges.append({"source": m, "target": imp, "attributes": {"disabled": True}})
# edges

In [ ]:
# Check to make sure edges are consistent with nodes

module_set = set(modules)
for e in edges:
    if e["source"] not in module_set:
        print("MISSING source", e)
    if e["target"] not in module_set:
        print("MISSING target", e)

In [ ]:
spec = {"nodes": nodes, "edges": edges}
# spec

In [ ]:
for node in spec["nodes"]:
    inputs = [e["source"] for e in edges if e["target"] == node["label"]]
    outputs = [e["target"] for e in edges if e["source"] == node["label"]]
    inputs_s = "\n".join("  " + i for i in inputs)
    outputs_s = "\n".join("  " + o for o in outputs)
    node["description"] = "inputs:\n%s\n\noutputs:\n%s" % (inputs_s, outputs_s)

In [ ]:
def remove_nodes(spec, remove):
    remove = set(remove)
    nodes = [n for n in spec["nodes"] if n["label"] not in remove]
    edges = [
        e
        for e in spec["edges"]
        if e["source"] not in remove and e["target"] not in remove
    ]
    return {"nodes": nodes, "edges": edges}

In [ ]:
# These are base modules that don't or shouldn't have dependencies
# to_remove = ['weave_types', 'op_args', 'errors', 'uris', 'box']

final_spec = remove_nodes(spec, ["errors"])
final_spec
plot(final_spec)